In [2]:
from behavior_and_time_alignment import beh_and_time_alignment
from session_preprocessing import ephys_opto_preprocessing, session_crosscorr
from opto_tagging import opto_plotting_session, opto_plotting_unit, opto_tagged_spike_stability
from opto_waveforms_preprocessing import opto_wf_preprocessing
import matplotlib.pyplot as plt
from utils.beh_functions import session_dirs
import numpy as np
import pandas as pd
import os
from utils.plot_utils import combine_pdf_big

In [3]:
session_list = ['behavior_758017_2025-02-04_11-57-38',
                'behavior_758017_2025-02-05_11-42-34',
                'behavior_758017_2025-02-06_11-26-14',
                'behavior_758017_2025-02-07_14-11-08']

In [4]:
session_list = ['behavior_751004_2024-12-19_11-50-37',
'behavior_751004_2024-12-20_13-26-11',
'behavior_751004_2024-12-21_13-28-28',
'behavior_751004_2024-12-22_13-09-17',
'behavior_751004_2024-12-23_14-20-03']


In [3]:
session_assets = pd.read_csv('/root/capsule/code/data_management/session_assets.csv')
session_list = session_assets['session_id']

In [4]:
aniIDs = ['751004']
session_list = [session for session in session_list if any(aniID in session for aniID in aniIDs)]
session_list

['behavior_751004_2024-12-19_11-50-37',
 'behavior_751004_2024-12-20_13-26-11',
 'behavior_751004_2024-12-21_13-28-28',
 'behavior_751004_2024-12-22_13-09-17',
 'behavior_751004_2024-12-23_14-20-03']

In [2]:
session_list = ['behavior_751766_2025-02-11_11-53-38',
                'behavior_751766_2025-02-14_11-37-11',
                'behavior_751769_2025-01-16_11-32-05',
                'behavior_751769_2025-01-17_11-37-39',
                'behavior_751769_2025-01-18_10-15-25',]

In [3]:
data_type = 'raw'
for session in session_list:
    pdf_dir = session_dirs(session)[f'opto_dir_fig_{data_type}']
    output_pdf = os.path.join(session_dirs(session)[f'opto_dir_{data_type}'], f'{session}_opto_tagging_png.pdf')
    combine_pdf_big(pdf_dir, output_pdf)

Combined PDF saved as: /root/capsule/scratch/751766/behavior_751766_2025-02-11_11-53-38/ephys/opto/raw/behavior_751766_2025-02-11_11-53-38_opto_tagging_png.pdf
Combined PDF saved as: /root/capsule/scratch/751766/behavior_751766_2025-02-14_11-37-11/ephys/opto/raw/behavior_751766_2025-02-14_11-37-11_opto_tagging_png.pdf
Combined PDF saved as: /root/capsule/scratch/751769/behavior_751769_2025-01-16_11-32-05/ephys/opto/raw/behavior_751769_2025-01-16_11-32-05_opto_tagging_png.pdf
Combined PDF saved as: /root/capsule/scratch/751769/behavior_751769_2025-01-17_11-37-39/ephys/opto/raw/behavior_751769_2025-01-17_11-37-39_opto_tagging_png.pdf
Combined PDF saved as: /root/capsule/scratch/751769/behavior_751769_2025-01-18_10-15-25/ephys/opto/raw/behavior_751769_2025-01-18_10-15-25_opto_tagging_png.pdf


In [6]:
from joblib import Parallel, delayed
# session_list = ['behavior_717121_2024-06-15_10-00-58',
#                 'behavior_716325_2024-05-31_10-31-14']
data_type = 'curated'
def process(session, data_type='raw'):
    # beh_and_time_alignment(session)
    ephys_opto_preprocessing(session, data_type, 'soma')
    # ephys_opto_crosscorr(session, data_type)
    # opto_wf_preprocessing(session, data_type, 'soma', load_sorting_analyzer=True)
    
    plt.close('all')
    opto_tagging_df = opto_plotting_session(session, data_type, 'soma', plot=True, resp_thresh=0.4, lat_thresh=0.02)
    plt.close('all')
    opto_tagged_spike_stability(session, data_type, 'soma', opto_tagging_df=opto_tagging_df)

# Parallel(n_jobs=3)(delayed(process)(session) for session in session_list)


In [7]:
process('behavior_716325_2024-05-31_10-31-14')

behavior_716325_2024-05-31_10-31-14


/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.0 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter sparsity from quality_metrics extension
  warnings.warn(f"Removing legacy parameter {p} from

compute_waveforms:   0%|          | 0/7025 [00:00<?, ?it/s]

/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.1 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.0 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/o

80 out of 359 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:386: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')


Merging file 0 out of 359
Merging file 50 out of 359
Merging file 100 out of 359
Merging file 150 out of 359
Merging file 200 out of 359
Merging file 250 out of 359
Merging file 300 out of 359
Merging file 350 out of 359
PDF files in '/root/capsule/scratch/716325/behavior_716325_2024-05-31_10-31-14/ephys/opto/raw/figures' merged into '/root/capsule/scratch/716325/behavior_716325_2024-05-31_10-31-14/ephys/opto/raw/behavior_716325_2024-05-31_10-31-14_opto_tagging.pdf' successfully.
16 out of 359 units pass quality control and opto tagging


/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.0 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter sparsity from quality_metrics extension
  warnings.warn(f"Removing legacy parameter {p} from

In [ ]:

Parallel(n_jobs=3)(delayed(process)(session) for session in session_list)

In [8]:
plt.close('all')
data_type = 'raw'
session = session_list[0]
opto_tagging_df = opto_plotting_session(session, data_type, 'soma', plot=True, resp_thresh=0.4, lat_thresh=0.02)
plt.close('all')
opto_tagged_spike_stability(session, data_type, 'soma', opto_tagging_df=opto_tagging_df)

/root/capsule/code/beh_ephys_analysis/opto_tagging.py:345: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')


95 out of 271 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:386: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')


Merging file 0 out of 271
Merging file 50 out of 271
Merging file 100 out of 271
Merging file 150 out of 271
Merging file 200 out of 271
Merging file 250 out of 271
PDF files in '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/raw/figures' merged into '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/raw/behavior_751004_2024-12-19_11-50-37_opto_tagging.pdf' successfully.
5 out of 271 units pass quality control and opto tagging


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:458: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  opto_tagging_df_focus = opto_tagging_df_sorted[(opto_tagging_df_sorted['opto_pass'] == True) & (opto_tagging_df['decoder_label'] != 'artifact') & (opto_tagging_df['decoder_label'] != 'noise')].copy()


In [ ]:
for session in session_list:
    ephys_opto_preprocessing(session, 'curated', 'soma')
    opto_wf_preprocessing(session, 'curated', 'soma', load_sorting_analyzer=False)
    opto_plotting_session(session, 'curated', plot=True)
    plt.close('all')

In [7]:
session = 'behavior_751004_2024-12-20_13-26-11'
data_type = 'raw'
target = 'soma'
target_unit_ids = [16]
# pass_df = opto_plotting_session(session, data_type, target, plot=True,  resp_thresh=0.5, lat_thresh=0.015, target_unit_ids=target_unit_ids)

In [ ]:
data_type = 'raw'
target = 'soma'
for session in session_list:
    combine_qc = opto_plotting_session(session, data_type, target, plot=True,  resp_thresh=0.5, lat_thresh=0.015)

In [8]:
session_dir = session_dirs(session)
opto_wf_csv = os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_waveform_metrics.csv')
opto_wf = pd.read_csv(opto_wf_csv)

In [7]:
pip install ipykernel